In [1]:
import numpy as np
import xarray as xr
import os
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider
import cartopy.crs as ccrs
from PIL import Image
import dask
# from datetime import datetime
import datetime
import pandas as pd
import timeit
from itertools import product
import sys

In [2]:
M = 0.8 #change later
reference_fuel_consumption = 0.7296 #who knows
reference_nox = 19.1835 #IDGAF
number_of_engines = 2 
temp_sea_level = 288.15 #kelvins
pres_sea_level = 101325 #pascals
gamma = 1.4
R = 287.05

In [3]:
#merge echam, accf and contrail datasets for each month into datasets representing whole year
dir_begin = "E:/Docs/studies/Delft/TestAnalysisSimulation/fromWebDrive/"
dir_mid = "DT00/"
# directory = os.fsencode("C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/AT20_optimal")
# directory_1percent = os.fsencode("C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/SOC_1pc")
# directory_soc = os.fsencode("C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/SOC_optimal")
# dir = "C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/AT20_optimal"
# dir_1percent = "C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/SOC_1pc"
# dir_soc = "C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/SOC_optimal"
directory = os.fsencode(dir_begin + dir_mid + "AT20_optimal")
directory_1percent = os.fsencode(dir_begin + dir_mid + "SOC_1pc")
directory_soc = os.fsencode(dir_begin + dir_mid + "SOC_optimal")
dir = dir_begin + dir_mid + "AT20_optimal"
dir_1percent = dir_begin + dir_mid + "SOC_1pc"
dir_soc = dir_begin + dir_mid + "SOC_optimal"
echam_files = []
accf_files = []
contrail_files = []
airtraf_files = []
airtraf_ac_co_files = []
airtraf_ac_1percent_files = []
airtraf_ac_soc_files = []
airtraf_gp_soc_files = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith("ECHAM5.nc"):
        echam_files.append(os.path.join(dir, filename))
    elif filename.endswith("accf_gp.nc"):
        accf_files.append(os.path.join(dir, filename))
    elif filename.endswith("contrail_gp.nc"):
        contrail_files.append(os.path.join(dir, filename))
    elif filename.endswith("airtraf_gp.nc"):
        airtraf_files.append(os.path.join(dir, filename))
    elif filename.endswith("airtraf_ac.nc"):
        airtraf_ac_co_files.append(os.path.join(dir, filename))
    else:
        continue
for file in os.listdir(directory_1percent):
    filename = os.fsdecode(file)
    if filename.endswith("airtraf_ac.nc"):
        airtraf_ac_1percent_files.append(os.path.join(dir_1percent, filename))
    else:
        continue
for file in os.listdir(directory_soc):
    filename = os.fsdecode(file)
    if filename.endswith("airtraf_ac.nc"):
        airtraf_ac_soc_files.append(os.path.join(dir_soc, filename))
    elif filename.endswith("airtraf_gp.nc"):
        airtraf_gp_soc_files.append(os.path.join(dir_soc, filename))
    else:
        continue
if echam_files:
    merged_echam = xr.open_mfdataset(echam_files, combine='nested', concat_dim='time')
if accf_files:
    merged_accf = xr.open_mfdataset(accf_files, combine='nested', concat_dim='time')
if contrail_files:
    merged_contrail = xr.open_mfdataset(contrail_files, combine='nested', concat_dim='time')
if airtraf_files:
    merged_airtraf = xr.open_mfdataset(airtraf_files, combine='nested', concat_dim='time')
if airtraf_ac_co_files:
    merged_airtraf_ac_co = xr.open_mfdataset(airtraf_ac_co_files, combine='nested', concat_dim='time')
if airtraf_ac_1percent_files:
    merged_airtraf_ac_1percent = xr.open_mfdataset(airtraf_ac_1percent_files, combine='nested', concat_dim='time')
if airtraf_ac_soc_files:
    merged_airtraf_ac_soc = xr.open_mfdataset(airtraf_ac_soc_files, combine='nested', concat_dim='time')
if airtraf_gp_soc_files:
    merged_airtraf_gp_soc = xr.open_mfdataset(airtraf_gp_soc_files, combine='nested', concat_dim='time')

#taking 2018 as the only time period
merged_echam = merged_echam.sel(time = slice("2018-01-01", "2018-12-31"))
merged_accf = merged_accf.sel(time = slice("2018-01-01", "2018-12-31"))
merged_contrail = merged_contrail.sel(time = slice("2018-01-01", "2018-12-31"))
merged_airtraf = merged_airtraf.sel(time = slice("2018-01-01", "2018-12-31"))
merged_airtraf_ac_co = merged_airtraf_ac_co.sel(time = slice("2018-01-01", "2018-12-31"))
merged_airtraf_ac_1percent = merged_airtraf_ac_1percent.sel(time = slice("2018-01-01", "2018-12-31"))
merged_airtraf_ac_soc = merged_airtraf_ac_soc.sel(time = slice("2018-01-01", "2018-12-31"))
merged_airtraf_gp_soc = merged_airtraf_gp_soc.sel(time = slice("2018-01-01", "2018-12-31"))

In [4]:
#aircrafts velocity based on its mach number
velocity_uncompressed = M*np.sqrt(gamma*R*merged_echam['tm1']) #m/s
#correction coefficients for fuel consumption
delta_uncompressed = merged_echam['tm1']*(1+0.2*M**2)**3.5 / temp_sea_level
theta_uncompressed = merged_echam['press']*(1+0.2*M**2)**3.5 / pres_sea_level
#actual fuel consumption per engine
f_a_i_uncompressed = reference_fuel_consumption*delta_uncompressed*np.sqrt(theta_uncompressed) #kg/s
#total fuel consumption
f_cr_uncompressed = f_a_i_uncompressed*number_of_engines #kg/s
#humidity correction coefficient
H_c_uncompressed = np.exp(-19*(merged_echam['qm1']-0.00634))
#nox emission corrected for cruise conditions
EINO_x_uncompressed = reference_nox*delta_uncompressed**0.4*theta_uncompressed**0.3*H_c_uncompressed #g/kg
#F-ATR20 for each emission in kelvins per kilometer
co2_atr20_perkm_uncompressed = merged_accf['atr20_co2']/velocity_uncompressed*f_cr_uncompressed*1000 #K/km
h2o_atr20_perkm_uncompressed = merged_accf['atr20_h2o']/velocity_uncompressed*f_cr_uncompressed*1000 #K/km
o3_atr20_perkm_uncompressed = merged_accf['atr20_o3']/velocity_uncompressed*f_cr_uncompressed*EINO_x_uncompressed*10**(-3)*1000 #K/km
ch4_atr20_perkm_uncompressed = merged_accf['atr20_ch4']/velocity_uncompressed*f_cr_uncompressed*EINO_x_uncompressed*10**(-3)*1000 #K/km
contr_atr20_perkm_uncompressed = merged_accf['atr20_contrail']*merged_contrail['potcov'] #?
total_atr20_perkm_uncompressed = co2_atr20_perkm_uncompressed + h2o_atr20_perkm_uncompressed + o3_atr20_perkm_uncompressed + ch4_atr20_perkm_uncompressed + contr_atr20_perkm_uncompressed

In [5]:
columns = ['month', 'day', 'time', 'i_cost', 'i_1p', 'i_climate', 'abs_comp', 'abs_opt', 'rel_comp', 'rel_opt']
gerben = pd.read_csv('gerben_output/aa_high_potential_list_sorted_rel_climate.txt', header=None, names=columns)

date_list = []
for i in range(len(gerben)):
    date_list.append(datetime.date(2018, gerben['month'].values[i], 1 + gerben['day'].values[i]))
index_list_cost = gerben['i_cost'].values
index_list_climate = gerben['i_climate'].values
cmb_arr = np.column_stack((date_list, index_list_climate))

In [6]:
mapped = total_atr20_perkm_uncompressed.interp(lev=[7, 9.141, 10.336, 11.270, 12.069, 12.759, 13.394, 14])

In [7]:
var = mapped.var(dim='lev')
var = xr.concat([var.sel(lon=slice(0,67.5)), var.sel(lon=slice(337.5, 357.1875))], dim="lon")
varcopy = var

In [23]:
columns_atr = ['Date', 'time', 'Flight index', 'ATRsum']
date = merged_airtraf_ac_soc.coords['time'].values
routes = np.arange(100)
lp1, lp2 = pd.core.reshape.util.cartesian_product([date, routes])
ATR_array = pd.DataFrame(dict(Date=lp1, time=0, Flight_index=lp2, ATRsum=float(0)))
ATR_array['Date'] = pd.to_datetime(ATR_array["Date"])
ATR_array

,Date,time,Flight_index,ATRsum
0,2018-01-01,0,0,0.0
1,2018-01-01,0,1,0.0
2,2018-01-01,0,2,0.0
3,2018-01-01,0,3,0.0
4,2018-01-01,0,4,0.0
...,...,...,...,...
36495,2018-12-31,0,95,0.0
36496,2018-12-31,0,96,0.0
36497,2018-12-31,0,97,0.0
36498,2018-12-31,0,98,0.0


In [9]:
lons = var.coords['lon'].values
lats = var.coords['lat'].values
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [0]:
lon_prop = merged_airtraf_ac_soc['routes_out'].isel(AirTraf_properties=0)
lat_prop = merged_airtraf_ac_soc['routes_out'].isel(AirTraf_properties=1)
varcopy_panda = var.to_dataframe(name='varcopy')
varcopy_panda.reset_index(inplace=True)

In [26]:
month = 7
# day = 7
for day in range(1, 8):
    ATR_array = pd.DataFrame(dict(Date=lp1, time=0, Flight_index=lp2, ATRsum=float(0)))
    ATR_array['Date'] = pd.to_datetime(ATR_array["Date"])
    for index, row in ATR_array.iterrows():
        if row['Date'].month == month and row['Date'].day == day:
            t = row["Date"]
            t = t.strftime('%Y-%m-%d')
            #removes the h-m-s time from the date format
            fid = row["Flight_index"]
            lon_fin = lon_prop.sel(time=t).isel(AirTraf_routes_out=fid)
            lat_fin = lat_prop.sel(time=t).isel(AirTraf_routes_out=fid)
            x = lon_fin.values
            y = lat_fin.values
            varcopy_panda_time = varcopy_panda.loc[(varcopy_panda['time'] == t)]
            summed_values = float(0)
            print(t, fid)
            for w in range(101):
                x_t = x[w]
                y_t = y[w]

                if x_t < (357.1875-360)/2:
                    x_t += 360
                lon_v = find_nearest(lons, x_t)
                lat_v = find_nearest(lats, y_t)
                valvar = float(varcopy_panda_time.loc[(varcopy_panda_time['lon'] == lon_v) & (varcopy_panda_time['lat'] == lat_v)]['varcopy'])
                summed_values += valvar

            # row['ATRsum']  = summed_values
            #the above method didn't work, I don't know why
            ATR_array.loc[(ATR_array['Date'] == t) & (ATR_array['Flight_index'] == fid), 'ATRsum']  = summed_values
            # ATR_array.at[index, 'ATRsum'] = summed_values

    save_path = f"partial_output/ATRsum-2018-{month:02d}-{day:02d}.csv"
    print(save_path)
    ATR_array.to_csv(save_path, index=False)


2018-07-01 0
2018-07-01 1
2018-07-01 2
2018-07-01 3
2018-07-01 4
2018-07-01 5
2018-07-01 6
2018-07-01 7
2018-07-01 8
2018-07-01 9
2018-07-01 10
2018-07-01 11
2018-07-01 12
2018-07-01 13
2018-07-01 14
2018-07-01 15
2018-07-01 16
2018-07-01 17
2018-07-01 18
2018-07-01 19
2018-07-01 20
2018-07-01 21
2018-07-01 22
2018-07-01 23
2018-07-01 24
2018-07-01 25
2018-07-01 26
2018-07-01 27
2018-07-01 28
2018-07-01 29
2018-07-01 30
2018-07-01 31
2018-07-01 32
2018-07-01 33
2018-07-01 34
2018-07-01 35
2018-07-01 36
2018-07-01 37
2018-07-01 38
2018-07-01 39
2018-07-01 40
2018-07-01 41
2018-07-01 42
2018-07-01 43
2018-07-01 44
2018-07-01 45
2018-07-01 46
2018-07-01 47
2018-07-01 48
2018-07-01 49
2018-07-01 50
2018-07-01 51
2018-07-01 52
2018-07-01 53
2018-07-01 54
2018-07-01 55
2018-07-01 56
2018-07-01 57
2018-07-01 58
2018-07-01 59
2018-07-01 60
2018-07-01 61
2018-07-01 62
2018-07-01 63
2018-07-01 64
2018-07-01 65
2018-07-01 66
2018-07-01 67
2018-07-01 68
2018-07-01 69
2018-07-01 70
2018-07-01 71
20